Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# Set Cover problem

See: https://en.wikipedia.org/wiki/Set_cover_problem

In [88]:
# from random import random, seed
# from itertools import product
import numpy as np
from numpy.typing import NDArray
from icecream import ic
from functools import cache


## Reproducible Initialization

If you want to get reproducible results, use `rng` (and restart the kernel); for non-reproducible ones, use `np.random`.

In [89]:
# The original value were too high for my computer to handle
# UNIVERSE_SIZE = 100_000
# NUM_SETS = 10_000
UNIVERSE_SIZE = 10_000
NUM_SETS = 1_000
DENSITY = 0.3

rng = np.random.Generator(np.random.PCG64([UNIVERSE_SIZE, NUM_SETS, int(NUM_SETS * DENSITY)]))

In [90]:
# DON'T EDIT THESE LINES!

SETS = np.random.random((NUM_SETS, UNIVERSE_SIZE)) < DENSITY
for s in range(UNIVERSE_SIZE):
    if not np.any(SETS[:, s]):
        SETS[np.random.randint(NUM_SETS), s] = True
COSTS = np.pow(SETS.sum(axis=1), 1.1)

## Helper Functions

In [91]:
def valid(solution):
    """Checks wether solution is valid (ie. covers all universe)"""
    return np.all(np.logical_or.reduce(SETS[solution]))


def cost(solution):
    """Returns the cost of a solution (to be minimized)"""
    return COSTS[solution].sum()

## Have Fun!

In [92]:
# A dumb solution of "all" sets
solution = np.full(NUM_SETS, True)
valid(solution), cost(solution)

(np.True_, np.float64(6675263.903467085))

In [93]:
# A random solution with random 50% of the sets
solution: NDArray[np.bool] = rng.random(NUM_SETS) < .5
valid(solution), cost(solution)

(np.True_, np.float64(3397163.3113611042))

### Note
There is some repetition for both solution but in this case I preferred to abstract the two method as they were completely independent, 
mainly for readability.

Also the results are cached as the `NUM_SETS` and `UNIVERSE_SIZE` are global variable that does not change

In [94]:
@cache
def greedy_set_cover() -> NDArray[np.bool]:
    """ Solves the set cover problem using a greedy algorithm 
    that selects the set that maximize the number of uncovered elements.
    
    Note: This is a naive solution that does not consider the added cost of each set.
    
    Returns:
        NDArray[np.bool]: Solution to the set cover problem
    """
    covered: set = set()

    # NUM_SETS.size bool array, initialized with False
    solution: NDArray[np.bool] = np.zeros(NUM_SETS, dtype=np.bool)

    # Create a copy of SETS to avoid modifying the original
    sets_copy: NDArray[np.bool] = SETS.copy()

    while len(covered) < UNIVERSE_SIZE:
        # Select the set that covers the most uncovered elements
        best_set_index: int = max(range(NUM_SETS), key=lambda i: len(set(np.where(sets_copy[i])[0]) - covered))
        solution[best_set_index] = True
        covered.update(np.where(sets_copy[best_set_index])[0])
        sets_copy[best_set_index] = np.zeros(UNIVERSE_SIZE, dtype=np.bool_)  # Mark the set as used by emptying it

    return solution

@cache
def greedy_set_cover_with_cost() -> NDArray[np.bool]:
    """ Solves the set cover problem using a greedy algorithm
    that selects the set that minimizes the cost per uncovered element.
    
    Note: The logic is similar to the algorithm used in the "greedy_set_cover" function , but the set selection
    is based on the cost per uncovered element.
    
    Returns:
        NDArray[np.bool]: Solution to the set cover problem
    """
    covered: set = set()

    # NUM_SETS.size bool array, initialized with False
    solution: NDArray[np.bool] = np.zeros(NUM_SETS, dtype=np.bool)

    # Create a copy of SETS to avoid modifying the original
    sets_copy: NDArray[np.bool] = SETS.copy()

    while len(covered) < UNIVERSE_SIZE:
        # Select the set that minimizes the cost per uncovered element
        best_set_index: int = min(
            range(NUM_SETS), 
            key=lambda i: COSTS[i] / len(set(np.where(sets_copy[i])[0]) - covered) if len(set(np.where(sets_copy[i])[0]) - covered) > 0 else float('inf')
        )
        
        solution[best_set_index] = True
        covered.update(np.where(sets_copy[best_set_index])[0])
        sets_copy[best_set_index] = np.zeros(UNIVERSE_SIZE, dtype=np.bool_)  # Mark the set as used by emptying it

    return solution



In [95]:
def solve_set_cover(method: str, print_and_return: bool | None = None) -> NDArray[np.bool]:
    """
    Solves the set cover problem using the specified method.
    One of the following methods can be used:
    - greedy
    - naive
    - greedy_with_cost
    - with_cost
    - cost
    
    Args:
        method (str): The method to use to solve the set cover problem.
        print_and_return (bool, optional): Whether to print and return the result. Defaults to None.

    Returns:
        NDArray[np.bool]: The solution to the set cover problem.
        
    Raises:
        ValueError: If an invalid method is specified.
    """
    naive_methods: list[str] = ["greedy", "naive"]
    cost_methods: list[str] = ["greedy_with_cost", "with_cost", "cost"]
    valid_methods: list[str] = naive_methods + cost_methods

    solution: NDArray[np.bool] = np.zeros(NUM_SETS, dtype=np.bool)
    
    method = method.lower().strip()  
        
    if method in naive_methods:
        solution = greedy_set_cover()
    elif method in cost_methods:
        solution = greedy_set_cover_with_cost()
    elif method not in valid_methods:
        raise ValueError(f"Invalid method: {method}. Use one of the following: {valid_methods}")
    
    if print_and_return:
        ic(method, valid(solution), cost(solution))
    return solution

In [99]:

naive_sol: NDArray[np.bool] = solve_set_cover("greedy", True)
cost_sol: NDArray[np.bool] = solve_set_cover("with_cost", True)

if valid(naive_sol) and valid(cost_sol):
    if cost(naive_sol) < cost(cost_sol):
        print(f"\n`greedy_set_cover` is better than `greedy_set_cover_with_cost` by a factor of ~{cost(cost_sol) / cost(naive_sol):.2f}")
    elif cost(naive_sol) == cost(cost_sol):
        print(f"\nBoth solutions have the same cost ~{cost(cost_sol):.2f}")
    elif cost(naive_sol) > cost(cost_sol):
        print(f"\n`greedy_set_cover_with_cost` is better than `greedy_set_cover` by a factor of ~{cost(naive_sol) / cost(cost_sol):.2f}")
else:
    raise ValueError("Invalid solution")

ic| method: 'greedy'
    valid(solution): np.True_
    cost(solution): np.float64(108563.1518081353)
ic| method: 'with_cost'
    valid(solution): np.True_
    cost(solution): np.float64(106009.81029575248)



`greedy_set_cover_with_cost` is better than `greedy_set_cover` by a factor of ~1.02
